<a href="https://colab.research.google.com/github/yogesh-bhattarai/GEN_AI/blob/main/finetune_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate

In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.0 MB/s eta 0:00:00


In [4]:
!pip install trl

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sou

In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

In [ ]:
def finetune_llamav2():
  data= load_dataset("timdettmers/openassistant-guanaco",split="train")
  tokenizer= AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
  tokenizer.pad_token= tokenizer.eos_token
  bnb_config= BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  model= AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code= True)
  model.config.use_cache=False
  model.config.pretraining.tp=1
  peft_config= LoraConfig(
      lora_alpha=32,
      lora_dropout=0.05,
      r=64,
      bias="none",
      task_type= "CAUSAL_LM"
  )
  trainingarguments= TrainingArguments(
      output_dir="llama2_finetuned_chatbot",
      per_device_eval_batch_size=12,
      gradient_accumulation_steps=4,
      optim="paged_adam_8bit",
      learning_rate= 2e-4,
      lr_shedular_type="Linear"
      save_strategy="epoch",
      logging_steps=10,
      num_train_epochs=1,
      max_steps=20,
      fp16= True,
      push_to_hub=True
  )